In [1]:
import pandas as pd

In [ ]:
# prendre le minimum de colonne pour faciliter le chargement du DF
df_principals = pd.read_csv("title.principals.tsv.gz", sep = "\t", usecols=['tconst', 'nconst', 'category'])
print('Chargement df_principals OK')
df_name = pd.read_csv("name.basics.tsv.gz", sep = "\t", usecols=['nconst', 'primaryName'])
print('Chargement df_name OK')
df_basics = pd.read_csv("title.basics.tsv.gz", sep = "\t", usecols=["tconst", "titleType", "runtimeMinutes", "startYear", "primaryTitle", "isAdult", "genres"])
print('Chargement df_basics OK')
df_ratings = pd.read_csv("title.ratings.tsv.gz", sep = "\t")
print('Chargement df_ratings OK')
df_akas = pd.read_csv("title.akas.tsv.gz", sep = "\t", usecols =("titleId", "title", "region", "language"))
print('Chargement df_akas OK')
df_tmdb = pd.read_csv("tmdb_full.csv", sep = ",", usecols=['backdrop_path', 'budget', 'homepage', 'id', 'imdb_id', 'popularity', 'poster_path', 'revenue', 'title', 'vote_average', 'vote_count'])
print('Chargement df_tmdb OK')

In [ ]:
# Merge mutliple sur la totalité des DF nécessaires aux éléments statistiques

movie_stats=pd.merge\
            (pd.merge # 4ème merge : le 3ème avec TMDB
                (pd.merge # 3ème merge : le 2nd avec ratings
                    (pd.merge # 2nd merge : le 1er avec name
                        (pd.merge # 1er merge : principals avec basics
                            (left=df_principals.loc[df_principals["category"].isin(["actor", "actress", "director", "writer"])], # uniquement les acteurs/trices, réalisateurs, et scénaristes 
                            right=df_basics.loc[df_basics["titleType"].isin(["movie", "tvSeries"])], # uniquement movie et tvSeries
                            how="inner",
                            left_on="tconst",
                            right_on="tconst"),
                        df_name,
                        how="inner",
                        left_on="nconst",
                        right_on="nconst"),
                    df_ratings,
                    how="left",
                    left_on='tconst',
                    right_on='tconst'),
                df_akas[df_akas['region'] == ('FR')], # uniquement sur la région de diffusion "FR"
                how="inner",
                left_on="tconst",
                right_on='titleId'),
            df_tmdb,
            how="left",
            left_on="tconst",
            right_on="imdb_id")
            
print('Merge des bases OK')

In [ ]:
# compléter les valeurs nulles d'averageRating avec celle de vote_average quand c'est possible
movie_stats['averageRating'] = movie_stats['averageRating'].fillna(movie_stats['vote_average'])

# compléter les valeurs nulles de numVotes avec celles de vote_count quand c'est possible
movie_stats['numVotes'] = movie_stats['numVotes'].fillna(movie_stats['vote_count'])
print('Complétion des valeurs IMDB par TMDB OK')

In [ ]:
# supprimer les lignes quand vote_average et averageRating sont vides
movie_stats.drop(movie_stats[(movie_stats['vote_average'].isna()) & (movie_stats['averageRating'].isna())].index, inplace=True)
print('Suppression des lignes vides de vote_average et averageRating Ok')

# Supprimer les colonnes devenues inutiles
movie_stats.drop(columns=['primaryTitle', 'title_y', 'imdb_id', 'vote_average', 'vote_count'], inplace=True)
print('Suppression des colonnes inutiles OK')

# basculer les colonnes en numérique et remplacer les \\N par 0 pour pouvoir traiter du numérique dans les stats
movie_stats['startYear'] = pd.to_numeric(movie_stats['startYear'].replace('\\N', 0)).astype('int16')
movie_stats['runtimeMinutes'] = pd.to_numeric(movie_stats['runtimeMinutes'].replace('\\N', 0)).astype('int16')
print('Bascule colonne en numérique OK')

In [ ]:
# renommer colonne "title_x" en "title"
movie_stats.rename(columns={"title_x": "title"}, inplace=True)
print('Renommage colonne title OK')

# suppression des doublons
movie_stats.drop_duplicates(inplace=True)
print('Suppression des doublons OK')

# rassembler les nconst en list par category (actor, actress, director, writer)
groupby_nconst = (
    movie_stats.groupby(["tconst", "category"])
    .agg({"nconst": list})  # Agréger les noms dans une liste
    .unstack("category")
    .add_suffix('_nconst')# Séparer par catégorie
    .fillna("")  # Remplacer les NaN par des chaînes vides
)
print('Group By nconst OK')

In [ ]:
# rassembler les primaryName en list par category (actor, actress, director, writer)
groupby_name = (
    movie_stats.groupby(["tconst", "category"])
    .agg({"primaryName": list})  # Agréger les noms dans une liste
    .unstack("category")
    .add_suffix('_name')# Séparer par catégorie
    .fillna("")  # Remplacer les NaN par des chaînes vides
)
print('Group By name OK')0

In [ ]:
# merger les 2 précédents
groupby_merged = pd.merge(left=groupby_nconst,
                right=groupby_name,
                how='inner',
                left_on='tconst',
                right_on='tconst')
print('Merge des groupby OK')

# ok pour supprimer niveau 1 colonne
groupby_merged.columns = groupby_merged.columns.droplevel()

# merger l'ensemble pour obtenir le DF final
movie_stats = pd.merge(left=movie_stats,
                  right=groupby_merged,
                  how='left',
                  left_on='tconst',
                  right_on='tconst'
                  )
print('Merge final OK')

In [ ]:
# supprimer les colonnes devenues inutiles
movie_stats = movie_stats.drop(columns=['nconst', 'category', 'primaryName'])
print('Suppression des colonnes inutiles OK')

# enfin, supprimer les tconst en doublons pour alléger le DF
movie_stats.drop_duplicates(subset='tconst', inplace=True)
print('Suppression des doublons OK')

In [ ]:
# convertir les colonnes de type liste en str pour faciliter la recherche
movie_stats['actor_name'] = movie_stats['actor_name'].apply(lambda x: ','.join(x))
movie_stats['actress_name'] = movie_stats['actress_name'].apply(lambda x: ','.join(x))
movie_stats['director_name'] = movie_stats['director_name'].apply(lambda x: ','.join(x))
movie_stats['writer_name'] = movie_stats['writer_name'].apply(lambda x: ','.join(x))
movie_stats['actor_nconst'] = movie_stats['actor_nconst'].apply(lambda x: ','.join(x))
movie_stats['actress_nconst'] = movie_stats['actress_nconst'].apply(lambda x: ','.join(x))
movie_stats['director_nconst'] = movie_stats['director_nconst'].apply(lambda x: ','.join(x))
movie_stats['writer_nconst'] = movie_stats['writer_nconst'].apply(lambda x: ','.join(x))
print('Conversion des listes en str OK')

movie_stats.to_csv("movie_stats.csv", index=False)
print('Conversion de movie_stats en .csv OK')

movie_stats.head(3)